In [27]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, util
import xml.etree.ElementTree as ET
import numpy as np

In [50]:
df = pd.read_excel('data/category_configuration_09-08-2022_08-08-01.xlsx', sheet_name = 'article_names_matching')
title_dict = dict(zip(df["Article Title"], df["Category 2"]))
title_dict.get("application")

'Non-derogation'

In [30]:
def get_article_text(article):
    
    text = ""

    for child in article:
        text += " "+child.text
    
    return text


In [55]:
def article_alignment(doc1_path, doc2_path):

    doc1, doc2 = ET.parse(doc1_path), ET.parse(doc2_path)
    root1, root2 = doc1.getroot(), doc2.getroot()
    try:
        body1, body2 = root1[1][2], root2[1][2]
    except:
        return
    num_pages = max(len(root1[1][2]), len(root2[1][2]))
    print('num_articles:', num_pages)
    model = SentenceTransformer('distilbert-base-nli-mean-tokens')
    #model = AutoModelForSequenceClassification.from_pretrained("roberta-large-mnli")
    article_dict = {}

    #if False:
    if "title" in body1[0].attrib and "title" in body2[0].attrib:
    #title included in article attri
        for article1 in body1:

            

            for article2 in body2:

                title1 = article1.get("title")
                title2 = article2.get("title")

                if title_dict.get(title1.lower()) == title_dict.get(title2.lower()):

                    text1_embd = model.encode(get_article_text(article1))
                    text2_embd = model.encode(get_article_text(article2))
                    scores = util.cos_sim(text1_embd, text2_embd)

                    if max(scores[0])>0.7:
                        article_dict[article1.get("num")] = article2.get("num")

    else:
    #title not included in article attri, use sentence similarity instead

        for article1 in body1:
            
            text1_embd = model.encode(get_article_text(article1))
            text2_list_embd = model.encode([get_article_text(article2) for article2 in body2])
            scores = util.cos_sim(text1_embd, text2_list_embd)
            maxi = max(scores[0])
            print(maxi)

            if maxi > 0.7:

                index = np.argmax(scores[0])
                article_dict[article1.get("num")] = body2[index].get("num")



    return article_dict

article_alignment('data/full data/t1989-9-canada-russian-federation-bit-1989.xml', 'data/full data/t1990-14-canada-czech-republic-bit-1990.xml')

num_articles: 14


{'I': 'I',
 'II': 'II',
 'III': 'III',
 'IV': 'IV',
 'V': 'V',
 'VI': 'VI',
 'VII': 'VII',
 'VIII': 'VIII',
 'IX': 'IX',
 'X': 'X',
 'XI': 'XI',
 'XIII': 'XII',
 'XIV': 'XIV'}